In [2]:
# import sqlite3
import numpy as np
# import mysql.connector
import pandas as pd
# import getpass
# import seaborn as sns
# import matplotlib.pyplot as plt
# import plotly
# plotly.tools.set_credentials_file(username='mainakchain', api_key='B9m4DVrYYmsoW3jDSiok')
# import plotly.plotly as py
# import plotly.graph_objs as go
# from plotly import __version__
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# import cufflinks as cf
# init_notebook_mode(connected=True)
# import plotly.figure_factory as ff
# from scipy import stats
from sklearn.utils import shuffle
from sklearn.neural_network import MLPRegressor
# from sklearn.neural_network import MLPClassifier
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
import datetime
import holidays
from sklearn.model_selection import cross_val_score
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import GridSearchCV
# from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
# from sklearn.svm import LinearSVR
from sklearn.svm import SVR
import operator

In [42]:
time = pd.read_csv('time_between_two_tolls_new.csv')

In [43]:
toll_freq = dict(time['toll_both'].value_counts())
time['toll_freq'] = time['toll_both']
time['toll_freq'] = time['toll_freq'].apply(lambda x: toll_freq[x] )
m = int(0.3 * time['toll_freq'].median())
time = time[(time['toll_freq'] > m)]
time.drop(['toll_freq'], axis=1, inplace=True)

In [44]:
time['loading_out_time'] = time['loading_out_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
time['ist_timestamp'] = time['ist_timestamp'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
time['hours']  = time['ist_timestamp'] - time['loading_out_time']
time['hours'] = time.hours.apply(lambda x:x.total_seconds()/3600)

In [45]:
time['distance_travelled'] = 0.0 #NL01N5129 CG04FB5095
time['time_taken'] = time['hours']
time['time_btn_tolls'] = 0  

In [46]:
new_time = pd.DataFrame(columns=time.columns.tolist())

In [47]:
for truck in time.vehicle_no.unique().tolist():
    for st_time in time.start_date.unique().tolist():
        df = time[(time.vehicle_no == truck) & (time.start_date == st_time)].sort_values('odometer_km')
        df.distance_travelled = df.odometer_km.diff()
        df.time_btn_tolls = df.hours.diff()
        new_time = new_time.append(df)

In [48]:
# new_time[new_time['time_btn_tolls'] == 0].shape
new_time = new_time.loc[new_time['distance_travelled'] != 0]

In [49]:
new_time = new_time.fillna(0)

In [177]:
new_time.to_csv('new_toll_time.csv',index=False)

In [110]:
toll_time = pd.read_csv('new_toll_time.csv')

In [111]:
toll_time.shape

(3386, 17)

In [112]:
# toll_time['loading_out_time'] = toll_time['loading_out_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
# toll_time['ist_timestamp'] = toll_time['ist_timestamp'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )

In [113]:
new_toll_time = toll_time.copy()

In [114]:
today_date = datetime.datetime.now()
today_date.strftime("%Y-%m-%d %H:%M:%S")
today = today_date.strftime("%Y-%m-%d %H:%M:%S")
new_toll_time['today'] = datetime.datetime.strptime(today, '%Y-%m-%d %H:%M:%S')

new_toll_time['start_date'] = new_toll_time['start_date'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['days_from_now'] = new_toll_time['today'] - new_toll_time['start_date'] 
new_toll_time['days_from_now'] = new_toll_time['days_from_now'].apply(lambda x: x.days)

new_toll_time['unloading_in_time'] = new_toll_time['unloading_in_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['avg_travel_time'] = new_toll_time['start_date'] + (new_toll_time['unloading_in_time'] - new_toll_time['start_date'])/2

new_toll_time['month_of_travel'] = new_toll_time['avg_travel_time'].apply(lambda x: x.month)

new_toll_time['ist_timestamp'] = new_toll_time['ist_timestamp'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
new_toll_time['time_of_travel'] = new_toll_time.ist_timestamp.apply(lambda x: x.hour)

new_toll_time['day_of_travel'] = new_toll_time['avg_travel_time'].apply(lambda x: x.weekday())

new_toll_time['loading_out_time'] = new_toll_time['loading_out_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['loading_out_time'] = new_toll_time['loading_out_time'].apply(lambda x: x.hour)

In [115]:
new_toll_time = new_toll_time[['toll_both','vehicle_no','loading_out_time','distance_travelled','time_of_travel','day_of_travel','month_of_travel','days_from_now','time_taken','time_btn_tolls']]

In [116]:
# toll_dict = {'Sergrarh':1,'Panikholi':2, 'Manguli':3, 'Gudipada (Old Gangapada)':4, 'Gurapalli':5, 'Bellupada':6, 'Madapam':7, 'Chilakapalem':8, 'Nathavalasa/ Vizianagaram':9, 'Laxmipuram':10, 'Vempadu':11, 'Krishnavaram':12, 'Kalaparru':13, 'Pottipadu':14, 'Kaza':15, 'Bolapalli':16, 'Sunambatti (Musunur)':17, 'Venkatachallam &#160;(Old Nellore)':18, 'Budanam':19, 'Gaddurur':20, 'Mulbagal':21, 'Hoskote':22, 'Attibele (BETL)':23, 'Aganampudi':24}

In [270]:
new_toll_time

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken,time_btn_tolls
0,Sergrarh,CG08L3445,10,0.00,1,4,12,9,38.391111,0.000000
1,Panikholi,CG08L3445,10,93.73,4,4,12,9,41.044167,2.653056
2,Manguli,CG08L3445,10,53.87,5,4,12,9,42.363889,1.319722
3,Gudipada (Old Gangapada),CG08L3445,10,56.09,6,4,12,9,43.947778,1.583889
4,Gurapalli,CG08L3445,10,87.68,10,4,12,9,47.070278,3.122500
5,Bellupada,CG08L3445,10,84.24,14,4,12,9,51.018889,3.948611
6,Madapam,CG08L3445,10,115.80,18,4,12,9,55.175278,4.156389
7,Chilakapalem,CG08L3445,10,27.17,18,4,12,9,55.961667,0.786389
8,Vempadu,CG08L3445,10,168.50,11,4,12,9,72.476111,16.514444
9,Krishnavaram,CG08L3445,10,70.47,14,4,12,9,75.802222,3.326111


In [119]:
dict_last_ist = {}
train_df = new_toll_time.copy()
test_df= pd.DataFrame(columns=['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now','time_taken','time_btn_tolls'])
choice = True
while(choice):
    today_date = datetime.datetime.now()
    vehicle = raw_input('Enter the vehicle no.: ')
    start = raw_input('Enter the start date in %Y-%m-%d %H:%M:%S format: ')
    start_delta = datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    
    loading_out = raw_input('Enter the loading out time in %Y-%m-%d %H:%M:%S format: ')
    loading_out_delta = datetime.datetime.strptime(loading_out, '%Y-%m-%d %H:%M:%S')
    
    a = []
    loop = input('Enter the number of tolls vehicle crossed SEQUENTIALLY: ')
    count = 0
    print '      '
    while (count != loop):
        b = []
        toll = raw_input('Toll both: ')
        distance = input('Distance travelled from previous toll in km: ')
        ist = raw_input('IST Timestamp in %Y-%m-%d %H:%M:%S format: ')
        ist_delta = datetime.datetime.strptime(ist, '%Y-%m-%d %H:%M:%S')
        b.extend([toll, vehicle, loading_out_delta.hour, distance, ist_delta.hour, ist_delta.weekday(), ist_delta.month,
                      (today_date - loading_out_delta).days, (ist_delta - loading_out_delta).total_seconds()/3600])
        a.append(b)
        dict_last_ist[vehicle] = ist_delta
        count+=1
    
    
    add_df = pd.DataFrame(a, columns= ['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now', 'time_taken'])
    
    add_df['time_btn_tolls'] = add_df['time_taken'].diff()
    add_df = add_df.fillna(0)
    
    train_df = train_df.append(add_df)
    
    test_toll = raw_input('Enter the numbers of tolls the vehicle shall take in the journey SEQUENTIALLY: ')
    test_toll_list = test_toll.split(',')
    test_toll_list = [str(a) for a in test_toll_list]

    a = []

    for toll_val in test_toll_list:
        b = []
        distance = toll_time['distance_travelled'][toll_time.toll_both == toll_val].median()
        ist_delta = loading_out_delta + timedelta(hours=toll_time['time_taken'][toll_time.toll_both == toll_val].median())
        b.extend([toll_val, vehicle, loading_out_delta.hour, distance, ist_delta.hour, ist_delta.weekday(), ist_delta.month,
                      (today_date - loading_out_delta).days, (ist_delta - loading_out_delta).total_seconds()/3600])

        a.append(b)
    add_test_df = pd.DataFrame(a, columns= ['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now','time_taken'])
    
    add_test_df['time_btn_tolls'] = add_test_df['time_taken'].diff()
    add_test_df = add_test_df.fillna(0)
    
    test_df = test_df.append(add_test_df)
    
    c = raw_input('Want to enter more vehicles?(y/n)')
    
    if (c == 'y' or c == 'Y'):
        continue
    elif (c == 'n' or c == 'N'):
        choice = False
        print 'bbye'
    else:
        print 'Wrong Choice! Exiting'
        choice = False

Enter the vehicle no.: OR04M532M
Enter the start date in %Y-%m-%d %H:%M:%S format: 2017-11-02 00:00:00
Enter the loading out time in %Y-%m-%d %H:%M:%S format: 2017-11-03 12:02:43
Enter the number of tolls vehicle crossed SEQUENTIALLY: 3
      
Toll both: Sergrarh
Distance travelled from previous toll in km: 0
IST Timestamp in %Y-%m-%d %H:%M:%S format: 2017-11-04 08:22:51
Toll both: Panikholi


KeyboardInterrupt: 

In [118]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [16]:
# train_df.to_csv('Ffinal_train_df_14Dec2017.csv', index=False)
# test_df.to_csv('Ffinal_test_df_14Dec2017.csv', index=False)

In [120]:
train_df = pd.read_csv('Ffinal_train_df_14Dec2017.csv')
test_df = pd.read_csv('Ffinal_test_df_14Dec2017.csv')

In [128]:
train_df.loc[train_df['time_btn_tolls'] == 0, 'time_btn_tolls']  = train_df.loc[train_df['time_btn_tolls'] == 0, 'time_taken']  

In [271]:
# new_toll_time_copy = new_toll_time.copy()
# new_toll_time_copy.loc[train_df['time_btn_tolls'] == 0, 'time_btn_tolls']  = new_toll_time_copy.loc[train_df['time_btn_tolls'] == 0, 'time_taken']  
# new_toll_time_copy.to_csv('new_toll_time_no_0.csv')

In [129]:
dict_last_ist = {'OR04M532M': datetime.datetime(2017, 11, 4, 12, 10, 12),
 'OR04M53MM': datetime.datetime(2017, 10, 23, 1, 35, 49)}

In [130]:
def find_toll_num(df):
    toll_dict = {}
    new_toll_dict = {}
    x = {}
    num = 1
    for toll in df.toll_both.unique().tolist():
        t = df['time_taken'][df.toll_both == toll].median()
        toll_dict[toll] = t
    
    for key, value in sorted(toll_dict.iteritems(), key=lambda (k,v): (v,k)):
        x[key] = value
        new_toll_dict[key] = num
        num+=1
        
    return new_toll_dict, x

In [131]:
new_toll_dict,x = find_toll_num(new_toll_time)

In [132]:
new_toll_dict

{'Bellupada': 6,
 'Bolapalli': 16,
 'Budanam': 19,
 'Chilakapalem': 9,
 'Gaddurur': 21,
 'Gudipada (Old Gangapada)': 4,
 'Gurapalli': 5,
 'Kalaparru': 14,
 'Kaza': 15,
 'Krishnavaram': 12,
 'Laxmipuram': 7,
 'Madapam': 8,
 'Manguli': 3,
 'Mulbagal': 20,
 'Nathavalasa/ Vizianagaram': 10,
 'Panikholi': 2,
 'Pottipadu': 13,
 'Sergrarh': 1,
 'Sunambatti (Musunur)': 18,
 'Tangutur': 17,
 'Vempadu': 11}

In [133]:
x

{'Bellupada': 54.63638888884999,
 'Bolapalli': 85.19555555560001,
 'Budanam': 94.75180555554999,
 'Chilakapalem': 60.240833333299996,
 'Gaddurur': 111.7175,
 'Gudipada (Old Gangapada)': 44.325833333350005,
 'Gurapalli': 47.752222222200004,
 'Kalaparru': 79.36486111114999,
 'Kaza': 80.6586111111,
 'Krishnavaram': 72.6825,
 'Laxmipuram': 58.4108333333,
 'Madapam': 58.9116666667,
 'Manguli': 42.618888888899995,
 'Mulbagal': 111.321388889,
 'Nathavalasa/ Vizianagaram': 64.30319444445,
 'Panikholi': 41.0441666667,
 'Pottipadu': 78.3761111111,
 'Sergrarh': 36.28625,
 'Sunambatti (Musunur)': 89.71486111109999,
 'Tangutur': 87.4901388889,
 'Vempadu': 68.81222222219999}

In [134]:
train_df.shape, test_df.shape

((3394, 10), (28, 10))

In [135]:
# test_df

In [136]:
# def change_toll_booth_to_dummy(df):
#     num = 1
#     dict_toll_both = {}
#     for toll in df['toll_both'].unique().tolist():
#         df['toll_both'] = df['toll_both'].replace({toll: num})
#         dict_toll_both[num] = toll
#         num += 1
#     return dict_toll_both

In [137]:
# dict_toll_both_train_df = change_toll_booth_to_dummy(train_df)
train_df.toll_both = train_df.toll_both.replace(new_toll_dict)
test_df.toll_both = test_df.toll_both.replace(new_toll_dict)

In [138]:
def change_vehicle_no_to_dummy(df):
    num = 1
    dict_vehicle_no = {}
    for truck in df['vehicle_no'].unique().tolist():
        df['vehicle_no'] = df['vehicle_no'].replace({truck: num})
        dict_vehicle_no[num] = truck
        num += 1
    return dict_vehicle_no

In [139]:
dict_vehicle_no_train_df = change_vehicle_no_to_dummy(train_df)

In [140]:
rev_dict_vehicle_no_test_df = dict((v,k) for k,v in dict_vehicle_no_train_df.iteritems())

In [141]:
test_df.vehicle_no = test_df.vehicle_no.replace(rev_dict_vehicle_no_test_df)

In [142]:
# train_df['virtual_avg_speed'] = train_df['distance_travelled'] / train_df['time_taken']
# test_df['virtual_avg_speed'] = test_df['distance_travelled'] / test_df['time_taken']

In [143]:
test_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken,time_btn_tolls
0,4,131,12,56.155,8,6,11,41,44.325833,0.000000
1,5,131,12,88.000,11,6,11,41,47.752222,3.426389
2,6,131,12,83.835,18,6,11,41,54.636389,6.884167
3,8,131,12,115.690,22,6,11,41,58.911667,4.275278
4,9,131,12,27.020,0,0,11,41,60.240833,1.329167
5,10,131,12,39.330,4,0,11,41,64.303194,4.062361
6,11,131,12,127.950,8,0,11,41,68.812222,4.509028
7,12,131,12,69.660,12,0,11,41,72.682500,3.870278
8,14,131,12,144.380,19,0,11,41,79.364861,6.682361
9,15,131,12,45.510,20,0,11,41,80.658611,1.293750


In [144]:
train_df.drop(['time_taken'], axis=1, inplace=True)

In [145]:
test_df.drop(['time_taken','time_btn_tolls'], axis=1, inplace=True)

In [146]:
test_df_copy = test_df.copy()

In [147]:
# MODEL BUILDING AND APPLICATION

In [148]:
train_X = train_df.drop(['time_btn_tolls'], axis=1)
train_y = train_df.time_btn_tolls

In [149]:
train_X.shape

(3394, 8)

In [150]:
X_train, X_test, y_train, y_test = train_test_split(train_X,train_y, test_size=0.3, random_state=42)

In [178]:
scale= MinMaxScaler()
X_train_scaled = scale.fit_transform(X_train)
X_test_scaled = scale.fit_transform(X_test)

In [327]:
# parameters = {"C": [1e1, 1e2, 1e3, 1e4],
#                                "gamma": np.logspace(-2, 2, 5)}

parameters = {
    'hidden_layer_sizes' : [(100,50),(200,50),(300,50)]
}

In [328]:
# reg_svr = SVR(kernel='rbf', gamma=0.1)

In [329]:
# grid = GridSearchCV(reg_svr, param_grid=parameters, n_jobs=-1,
#                    scoring='r2',
#                    verbose=1)
grid = GridSearchCV(reg_svr, cv=5,
                   param_grid=parameters, verbose=1, n_jobs=-1, scoring='r2')

In [274]:
grid_result = grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 10.1min finished


In [276]:
print("Best parameters set found on development set:")
print(grid_result.best_estimator_)
print("Grid scores on development set:")
for parameters, mean_score, scores in grid_result.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
          % (mean_score, scores.std() / 2, parameters))

Best parameters set found on development set:
SVR(C=100.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1.0,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
Grid scores on development set:
0.194 (+/-0.014) for {'C': 10.0, 'gamma': 0.01}
0.275 (+/-0.015) for {'C': 10.0, 'gamma': 0.10000000000000001}
0.307 (+/-0.012) for {'C': 10.0, 'gamma': 1.0}
0.202 (+/-0.017) for {'C': 10.0, 'gamma': 10.0}
0.041 (+/-0.017) for {'C': 10.0, 'gamma': 100.0}
0.269 (+/-0.015) for {'C': 100.0, 'gamma': 0.01}
0.288 (+/-0.016) for {'C': 100.0, 'gamma': 0.10000000000000001}
0.321 (+/-0.014) for {'C': 100.0, 'gamma': 1.0}
-0.037 (+/-0.036) for {'C': 100.0, 'gamma': 10.0}
0.057 (+/-0.029) for {'C': 100.0, 'gamma': 100.0}
0.283 (+/-0.016) for {'C': 1000.0, 'gamma': 0.01}
0.292 (+/-0.015) for {'C': 1000.0, 'gamma': 0.10000000000000001}
0.263 (+/-0.020) for {'C': 1000.0, 'gamma': 1.0}
-0.309 (+/-0.088) for {'C': 1000.0, 'gamma': 10.0}
0.057 (+/-0.029) for {'C': 1000.0, 'gamma': 10

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning:

The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20



In [210]:
reg_mlp = MLPRegressor(hidden_layer_sizes=(500,), activation='logistic', alpha=8, solver='lbfgs', random_state=42)

In [215]:
# reg_svr = SVR(C=100.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1.0,
#   kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
reg_mlp.fit(X_train_scaled, y_train)
reg_mlp.score(X_test_scaled, y_test)

0.36404290493138936

In [211]:
scores = cross_val_score(reg_mlp, X_train_scaled, y_train, scoring='r2', n_jobs=-1)

In [212]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.56 (+/- 0.04)


In [269]:
# reg_mlp.fit(X_train_scaled, y_train)
# reg_mlp.score(X_test_scaled, y_test) * 100

1.1986647608093981

In [270]:
# scores = cross_val_score(reg_mlp, X_train_scaled, y_train, scoring='r2', n_jobs=-1)
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.35 (+/- 0.05)


In [213]:
scale = MinMaxScaler()
X_predict_scaled = scale.fit_transform(test_df)

In [216]:
predictions = reg_mlp.predict(X_predict_scaled)

In [217]:
show_df = test_df.copy()
show_df['predicted'] = predictions

In [246]:
# show_df['predict_diff'] = show_df['predicted'].diff()

In [282]:
# def med_toll_time_taken(toll):
#     return train_df['time_taken'][train_df['toll_both'] == toll ].median()

In [283]:
# def mean_toll_time_taken(toll):
#     return train_df['time_taken'][train_df['toll_both'] == toll ].mean()

In [284]:
# show_df['med_time'] = show_df['toll_both']
# show_df['med_time'] = show_df.med_time.apply(lambda x:med_toll_time_taken(x))
# show_df['mean_time'] = show_df['toll_both']
# show_df['mean_time'] = show_df.mean_time.apply(lambda x:mean_toll_time_taken(x))

In [285]:
# show_df['avg_pred_med'] = (show_df['predicted'] + show_df['med_time']) / 2

In [286]:
# x = train_df
# x['avg_speed'] = x.distance_travelled/ x.time_taken
# x = x[x.vehicle_no == 25].sort_values('distance_travelled')
# x['avg_speed'] = x.distance_travelled/(x.time_taken - 40.88)
# x

In [287]:
# train_df[train_df['vehicle_no'] == 45]

In [288]:
# show_df['avg_speed_pred'] = show_df['distance_travelled']/show_df['predicted']
# show_df['avg_speed_med'] = show_df['distance_travelled']/show_df['med_time']

In [289]:
# show_df['test_pred'] = show_df['predicted'] + show_df.avg_speed_pred

In [333]:
# show_df['round_pred'] = show_df['predicted'].round()

In [291]:
# show_df['median_input'] =  test_df_copy['time_taken']

In [292]:
# show_df.drop(['virtual_avg_speed','predict_diff','mean_time','avg_pred_med','avg_speed_med','avg_speed_pred'], axis=1, inplace=True)

In [348]:
# rev_dict_vehicle_no_test_df['OR04M532M']

131

In [221]:
dict_last_ist = {'OR04M532M': datetime.datetime(2017, 11, 4, 12, 10, 12),
 'OR04M53MM': datetime.datetime(2017, 10, 23, 1, 35, 49)}

In [222]:
# for num, truck in dict_vehicle_no_train_df.iteritems():
list_truck = []
for truck, ist in dict_last_ist.iteritems():
    list_truck.append(truck)
    
for truck in list_truck:
    i = rev_dict_vehicle_no_test_df[truck]
    dict_last_ist[i] = dict_last_ist.pop(truck)

In [223]:
dict_last_ist

{131: datetime.datetime(2017, 11, 4, 12, 10, 12),
 132: datetime.datetime(2017, 10, 23, 1, 35, 49)}

In [224]:
show_df['eta'] = show_df['predicted']

In [225]:
x = datetime.datetime(2017, 11, 4, 12, 10, 12)
x + datetime.timedelta(hours=8.9)

datetime.datetime(2017, 11, 4, 21, 4, 12)

In [226]:
show_df['eta'][show_df['vehicle_no'] == 131]

0      7.479163
1      4.134102
2      3.622545
3      8.032359
4     11.896269
5      4.159764
6     11.399508
7      4.368015
8     14.193812
9      2.664515
10     6.779628
11     4.750822
12     4.782828
13     9.598972
14    23.749238
Name: eta, dtype: float64

In [253]:
show_df['time_taken'] = 0
for truck, ist in dict_last_ist.iteritems():
    show_df['time_taken'][show_df['vehicle_no'] == truck] = show_df['predicted'][show_df['vehicle_no'] == truck].cumsum()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [228]:
for truck, ist in dict_last_ist.iteritems():
    print ist

2017-10-23 01:35:49
2017-11-04 12:10:12


In [229]:
# def get_eta(ist, x):
#     y = ist + datetime.timedelta(hours=x)
#     return y

In [248]:
show_df.dtypes

toll_both                       int64
vehicle_no                      int64
loading_out_time                int64
distance_travelled            float64
time_of_travel                  int64
day_of_travel                   int64
month_of_travel                 int64
days_from_now                   int64
predicted                     float64
eta                    datetime64[ns]
time_taken            timedelta64[ns]
dtype: object

In [245]:
for truck, ist in dict_last_ist.iteritems():
    show_df['eta'][show_df['vehicle_no'] == truck] = str(ist)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [247]:
show_df['eta'] = show_df['eta'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )

In [262]:
toll_copy = toll_time.copy()
toll_copy['toll_both'] = toll_copy['toll_both'].replace(new_toll_dict)

In [266]:
toll_copy[toll_copy['toll_both'] == 21].time_taken.median()

111.7175

In [249]:
show_df['eta'] = show_df['eta'] + show_df['time_taken']

In [251]:
# show_df.drop(['time_taken'], axis=1, inplace=True)

In [267]:
train_df.tail(10)

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_btn_tolls
3384,14,130,23,146.31,0,3,11,25,7.129167
3385,13,130,23,21.61,0,3,11,25,0.483333
3386,1,131,12,0.00,8,5,11,41,20.335556
3387,2,131,12,94.90,10,5,11,41,2.322500
3388,3,131,12,54.20,12,5,11,41,1.466667
3389,1,132,21,0.00,17,6,10,53,20.183611
3390,2,132,21,94.30,19,6,10,53,2.391667
3391,3,132,21,54.50,21,6,10,53,1.594444
3392,4,132,21,56.09,23,6,10,53,2.156667
3393,5,132,21,88.60,1,0,10,53,2.258889


In [254]:
show_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,predicted,eta,time_taken
0,4,131,12,56.155,8,6,11,41,7.479163,2017-11-04 19:38:56.986745,7.479163
1,5,131,12,88.000,11,6,11,41,4.134102,2017-11-04 23:46:59.753713,11.613265
2,6,131,12,83.835,18,6,11,41,3.622545,2017-11-05 03:24:20.916424,15.235810
3,8,131,12,115.690,22,6,11,41,8.032359,2017-11-05 11:26:17.408600,23.268169
4,9,131,12,27.020,0,0,11,41,11.896269,2017-11-05 23:20:03.977875,35.164438
5,10,131,12,39.330,4,0,11,41,4.159764,2017-11-06 03:29:39.126488,39.324202
6,11,131,12,127.950,8,0,11,41,11.399508,2017-11-06 14:53:37.355313,50.723710
7,12,131,12,69.660,12,0,11,41,4.368015,2017-11-06 19:15:42.209151,55.091725
8,14,131,12,144.380,19,0,11,41,14.193812,2017-11-07 09:27:19.931264,69.285536
9,15,131,12,45.510,20,0,11,41,2.664515,2017-11-07 12:07:12.184931,71.950051
